In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import warnings
warnings.filterwarnings("ignore")
import TinySleepNet
import wandb
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import DatasetsPaco

In [2]:
# wandb.init()

In [3]:
binary_labels = True
channels = ['F4']
batch_size = 32

In [4]:
prueba = DatasetsPaco.EDFData_GEN_TF("/home/jorgevi/SSD_IA3/Data/PSG1.edf", channels=channels)

In [5]:
def our_generator():
    for x, y in prueba:
      yield x, y

dataset = tf.data.Dataset.from_generator(our_generator, (tf.float64, tf.int32)).batch(32).shuffle(64) # tf.int16 => tf.int32, tf.float32 => tf.float64

In [6]:
next(iter(dataset))

(<tf.Tensor: shape=(32, 15360, 1), dtype=float64, numpy=
 array([[[-1.92014534],
         [-1.61724675],
         [-1.59561114],
         ...,
         [ 1.5631884 ],
         [ 1.54155279],
         [ 1.08720491]],
 
        [[ 1.62809524],
         [ 1.95262944],
         [ 1.77954454],
         ...,
         [ 1.00066246],
         [ 0.43813651],
         [ 0.50304335]],
 
        [[ 0.30832283],
         [-0.34074557],
         [-0.47055925],
         ...,
         [-0.8816359 ],
         [-0.75182222],
         [-0.38401679]],
 
        ...,
 
        [[ 0.61122142],
         [ 0.24341599],
         [ 0.50304335],
         ...,
         [-1.55233991],
         [-0.96817835],
         [-0.83836467]],
 
        [[ 0.04869547],
         [ 0.02705986],
         [-0.10275382],
         ...,
         [ 0.67612826],
         [ 1.43337472],
         [ 2.55842662]],
 
        [[ 2.88296082],
         [ 1.93099383],
         [ 1.15211175],
         ...,
         [ 1.3901035 ],
         [ 1.

train data

In [4]:
# train_dataset = DatasetsPaco.EDFData_TF_old("/home/jorgevi/SSD_IA3/Data/PSG1.edf", batch_size=batch_size, channels=channels,
#                                                       binary_labels=binary_labels)
# labels_b = []
# for a, b in train_dataset:
#     labels_b.extend(b)

# X_binary_std = []
# for X, Y in train_dataset:
#     X_binary_std.append(X)
# X_binary_std = np.vstack(X_binary_std)

# tftraindatasetmemo = tf.data.Dataset.from_tensor_slices(
#     (tf.convert_to_tensor(X_binary_std), tf.convert_to_tensor(labels_b))).batch(32).shuffle(64)

# for x, y in train_dataset:
#     print(x, y)
#     break

test data

In [12]:
# test_dataset = DatasetsPaco.EDFData_TF_old("/home/jorgevi/SSD_IA3/Data/PSG5.edf", batch_size=batch_size, channels=channels,
#                                                      binary_labels=binary_labels)
# labels_test = []
# for a, b in test_dataset:
#     labels_test.extend(b)

# X_binary_std_test = []
# for X, Y in test_dataset:
#     X_binary_std_test.append(X.numpy())
# X_binary_std_test = np.vstack(X_binary_std_test)

# tftestdatasetmemo = tf.data.Dataset.from_tensor_slices(
#     (tf.convert_to_tensor(X_binary_std_test), tf.convert_to_tensor(labels_test))).batch(32).shuffle(64)

model train and validation

In [13]:
sr = int(prueba.sampling_rate)
num_epochs = 10
if binary_labels:
    classes = 2
else:
    classes = 5

In [14]:
model = TinySleepNet.TinySleepNet(sr, len(channels), classes)

Keep results for plotting

In [15]:
train_loss_results = []
train_accuracy_results = []
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [16]:
for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    for x, y in dataset:
        with tf.GradientTape() as tape:
            y_ = model(x)
            loss = loss_fn(y, y_)
        grads = tape.gradient(loss, model.trainable_variables)
        # wandb.log({'gradients': grads})
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Track progress
        epoch_loss_avg.update_state(loss)  # loss = loss.numpy()
        epoch_accuracy.update_state(y, y_)

    # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    # wandb.log({'epoch_loss': epoch_loss_avg.result().numpy(), 'epoch_accuracy': epoch_accuracy.result().numpy()})
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Epoch 000: Loss: 0.571, Accuracy: 76.190%
Epoch 001: Loss: 0.801, Accuracy: 52.154%
Epoch 002: Loss: 0.670, Accuracy: 72.109%
Epoch 003: Loss: 0.640, Accuracy: 72.109%
Epoch 004: Loss: 0.613, Accuracy: 72.109%
Epoch 005: Loss: 0.613, Accuracy: 71.882%
Epoch 006: Loss: 0.606, Accuracy: 71.995%
Epoch 007: Loss: 0.603, Accuracy: 71.769%
Epoch 008: Loss: 0.603, Accuracy: 71.882%
Epoch 009: Loss: 0.598, Accuracy: 71.995%


In [17]:
# model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
# history = model.fit(dataset, epochs=num_epochs, batch_size=32)
# model.summary()

# plt.plot(history.history['acc'])
# plt.plot(history.history['loss'])
# plt.savefig('train')

# ValueError: as_list() is not defined on an unknown TensorShape.

In [18]:
# test_accuracy = tf.keras.metrics.Accuracy()
# for x, y in tftestdatasetmemo:
#     logits = model(x)
#     predictions = tf.argmax(logits, axis=1, output_type=tf.int32)
#     test_accuracy(predictions, y)
# print("Test set accuracy: {:.3%}".format(test_accuracy.result()))